General DESI related packages

In [18]:
# general packages
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import specsim
import specsim.quickspecsim as qspecsim
import specsim.instrument as inst
import specsim.source as src
import specsim.config as conf
import specsim.fiberloss as floss
import specsim.observation as obs
import specsim.simulator as sim
import specsim.atmosphere as atm

from desisim import io
from desisim import obs

import astropy.units as u
import astropy.table

from scipy.interpolate import interp1d

In [20]:
# run the specsim.simulator with a configuration at "model"
def desi_model(model):
    return sim.Simulator(config_path+model+'.yaml')

num_source   = 1
DEBUG        = True
desi_configs = ['desi-blur', 'desi-blur-offset', 'desi-blur-offset-scale', 'desi-blur-offset-scale-stochastic']
config_path  = '/home/tyapici/Projects/DESI_projects/dithering/config/'
desi_models  = {}

# generate all relevant configurations for future use
for config in desi_configs:
    desi_models[config] = desi_model(config)

# default model is set to desi+blur+offset
curr_desi_model_string = 'desi-blur-offset'
curr_desi_model = desi_models[curr_desi_model_string]

# function to generate the wavelength values
def create_wlen_grid(num_wlen=11, desi=curr_desi_model):
    wavelength = desi.simulated['wavelength']
    wlen_unit = wavelength.unit
    return np.linspace(wavelength.data[0], wavelength.data[-1], num_wlen) * wlen_unit

# function to generate mock sources
def generate_source(nsrc=1, disk_fraction=1., bulge_fraction=0., vary='', seed=23, minormajor=[1,1]):
    gen = np.random.RandomState(seed)
    varied = vary.split(',')
    source_fraction = np.tile([disk_fraction, bulge_fraction], (nsrc, 1))
    source_half_light_radius = np.tile([0.45, 1.0], (nsrc, 1))
    source_minor_major_axis_ratio = np.tile(minormajor, (nsrc, 1))
    #if 'pa' in varied:
    source_position_angle = 360. * gen.uniform(size=(nsrc, 2))
    #else:
    #    source_position_angle = np.tile([0., 0.], (nsrc, 1))
    return source_fraction, source_half_light_radius, source_minor_major_axis_ratio, source_position_angle

round and elliptic sources has been generated


In [3]:
# function to generate the positions of the fibers.
# TODO: it is done with random positions, need to change with the actual positions
def generate_fiber_positions(nfiber=5000, seed=13, desi=curr_desi_model, isRandom=True):
    if isRandom:
        gen = np.random.RandomState(seed)
        focal_r = (np.sqrt(gen.uniform(size=nfiber)) * desi.instrument.field_radius)
        phi = 2 * np.pi * gen.uniform(size=nfiber)
    else:
        # this part is broken. needs some fixing
        gen = np.random.RandomState(seed)
        focal_r = (np.sqrt(gen.uniform(0, 0.001, size=nfiber)) * desi.instrument.field_radius)
        phi = 2 * np.pi * gen.uniform(size=nfiber)
    return np.cos(phi) * focal_r, np.sin(phi) * focal_r

In [4]:
# function to plot the fiber positions nicely
def plot_fiber_positions(focal_x, focal_y, save=None, desi=curr_desi_model, select=None):
    plt.figure(figsize=(8, 8))
    x, y = focal_x.to(u.mm).value, focal_y.to(u.mm).value
    plt.plot(x, y, '.')
    if select is not None:
        plt.scatter(x[select], y[select], s=100)
    r = 1.05 * desi.instrument.field_radius.to(u.mm).value
    plt.xlim(-1.01 * r, +1.01 * r)
    plt.ylim(-1.01 * r, +1.01 * r)
    plt.gca().set_aspect('equal', 'datalim')
    plt.gca().add_artist(plt.Circle((0, 0), r, color='r', fill=False))
    plt.axis('off')
    if save:
        plt.savefig(save)
    else:
        plt.show()

In [5]:
def calculateFiberLoss(wlen_grid, desi=curr_desi_model):
    calc = floss.GalsimFiberlossCalculator(desi.instrument.fiber_diameter.to(u.um).value,
                                           wlen_grid.to(u.Angstrom).value,
                                           num_pixels=16, oversampling=32, moffat_beta=3.5)
    return calc

In [6]:
def get_fiberloss(source_fraction, source_half_light_radius, source_minor_major_axis_ratio,
                  source_position_angle, x, y, wlen_grid, seeing=1.1*u.arcsec, desi=curr_desi_model):
    # Subtract instrumental PSF.
    #Jacoby = 0.219
    #seeing = 2.35482 * np.sqrt((seeing.to(u.arcsec).value/2.35482) ** 2 - Jacoby**2) * u.arcsec
    # Tabulate seeing.
    #desi.atmosphere.seeing_fwhm_ref = seeing
    seeing_fwhm = desi.atmosphere.get_seeing_fwhm(wlen_grid).to(u.arcsec).value
    # Calculate optics.
    # NOTE: TODO: may add dithering stuff here
    
    scale, blur, offset = desi.instrument.get_focal_plane_optics(x, y, wlen_grid)
    # Do the fiberloss calculations.
    calc = calculateFiberLoss(wlen_grid, desi=desi)
    return calc.calculate(seeing_fwhm,
                          scale.to(u.um / u.arcsec).value, offset.to(u.um).value,
                          blur.to(u.um).value,
                          source_fraction, source_half_light_radius,
                          source_minor_major_axis_ratio, source_position_angle)
    # there is something going wrong here with the source position angle
    # no matter what it is, the plots look the same for different sources at different locations

In [7]:
def get_desimodel_fiberloss(obj_type='lrg'):
    path = os.path.join(os.environ['DESIMODEL'], 'data', 'throughput',
                        'fiberloss-{0}.dat'.format(obj_type))
    t = astropy.table.Table.read(path, format='ascii', names=['wlen', 'accept'])
    return t

In [8]:
def plot_fiberloss(floss, cmap_name='viridis', alpha=0.1, ylim=(None, None),
                   desi=curr_desi_model, desimodel_type=None, save=None, overlay=False):
    if not overlay:
        plt.figure(figsize=(5, 4))
    wlen = wlen_grid.value
    rvalues = np.sqrt(focal_x ** 2 + focal_y ** 2)
    norm = matplotlib.colors.Normalize(
        vmin=0., vmax=desi.instrument.field_radius.to(u.mm).value)
    sm = matplotlib.cm.ScalarMappable(norm=norm, cmap=plt.get_cmap(cmap_name))
    for f, r in zip(floss, rvalues):
        color = sm.to_rgba(r)
        p = plt.plot(wlen, f, ls='-', c=color, alpha=0.2)
    plt.plot(wlen, np.median(floss, axis=0), 'k-', lw=2, label='median')
    if desimodel_type:
        t = get_desimodel_fiberloss(desimodel_type)
        plt.plot(t['wlen'], t['accept'], 'r--', lw=2, label='desimodel')
    if not overlay:
        plt.ylim(*ylim)
        # Very useful trick from http://stackoverflow.com/posts/11558629/revisions
        sm._A = []
        plt.colorbar(sm).set_label('Radius [mm]')
        plt.xlabel('Wavelength [Angstrom]')
        plt.ylabel('Fiber accepetance fraction')
        plt.xlim(wlen[0], wlen[-1])
        plt.tight_layout()
    if save:
        plt.savefig(save)

In [9]:
def plot_one_fiberloss(floss, ix, iy, wlen_grid, cmap_name='viridis', alpha=0.1, ylim=(None, None),
                       desi=curr_desi_model, desimodel_type=None, save=None, overlay=False):
    wlen = wlen_grid.value
    line_median, = plt.plot(wlen, np.median(floss, axis=0), 'k-', lw=2, label='dx=%d dy=%d'%(ix*5, iy*5), alpha=0.1)
    return line_median

In [46]:
def run(fiber_pos_seed, focal_x, focal_y, source_fraction, source_half_light_radius, 
        source_minor_major_axis_ratio, source_position_angle, append_name):
    # generate the wavelengths array
    wlen_grid = create_wlen_grid()
    print("wavelength grid has been generated")
    
    # plot the position of fibers
    # number of fibers needs to be equal to the number of sources
    # currently, there is only one source (that makes it easy to run a test)
    # TODO: make the whole script working for more than one fiber 
    if DEBUG:
        plot_fiber_positions(focal_x, focal_y, save="figures/fiber_pos_%d.png"%fiber_pos_seed)
        print("fiber position has been plotted")

    desi = curr_desi_model
    source_loss = get_fiberloss(source_fraction, source_half_light_radius, 
                                source_minor_major_axis_ratio,source_position_angle, 
                                focal_x, focal_y, wlen_grid, desi=desi)
    print("fiber loss calculations finished")
    
    x = []
    y = []
    acceptance_at_3550 = []
    acceptance_at_6700 = []
    acceptance_at_9850 = []
    expected_acceptance_at_3550 = []
    expected_acceptance_at_6700 = []
    expectedacceptance_at_9850 = []
    diff_acceptance_at_3550 = []
    diff_acceptance_at_6700 = []
    diff_acceptance_at_9850 = []
    num_dither = 4
    dither = np.linspace(-10, 10, num_dither)
    plt.clf()
    for ix in range(num_dither):
        for iy in range(num_dither):
            source_loss = get_fiberloss(source_fraction, source_half_light_radius, 
                                        source_minor_major_axis_ratio,source_position_angle, 
                                        focal_x+dither[ix]*5*u.mm, focal_y+dither[iy]*5.*u.mm, 
                                        wlen_grid, desi=desi)
            x.append(dither[ix]*5)
            y.append(dither[iy]*5)
            acceptance_at_3550.append(np.array(source_loss).flatten()[0])
            acceptance_at_6700.append(np.array(source_loss).flatten()[5])
            acceptance_at_9850.append(np.array(source_loss).flatten()[-1])
            if ix==0 and iy==0:
                center_acceptance_at_3550 = np.array(source_loss).flatten()[0]
                center_acceptance_at_6700 = np.array(source_loss).flatten()[5]
                center_acceptance_at_9850 = np.array(source_loss).flatten()[-1]
            l1 = plot_one_fiberloss(source_loss, ix, iy, 
                                    wlen_grid, ylim=(0.18, 0.68), desimodel_type='qso')
    diff_acceptance_at_3550 = acceptance_at_3550 - center_acceptance_at_3550
    diff_acceptance_at_6700 = acceptance_at_6700 - center_acceptance_at_6700
    diff_acceptance_at_9850 = acceptance_at_9850 - center_acceptance_at_9850
    
    t = get_desimodel_fiberloss('qso')
    xs = np.array(t['wlen'])
    ys = np.array(t['accept'])
    model_acceptance_func = interp1d(xs, ys, fill_value='extrapolate')
    expected_acceptance_at_3550 = model_acceptance_func(3550)
    expected_acceptance_at_6700 = model_acceptance_func(6700)
    expected_acceptance_at_9850 = model_acceptance_func(9850)
    line_model, = plt.plot(t['wlen'], t['accept'], 'r--', lw=2, label='desimodel')
    # TODO: depending on the source type the filenames here and after should be changed.
    plt.savefig("figures/ugly_plot_%s_%d_%s.png"%(curr_desi_model_string, fiber_pos_seed, append_name))
    print("arrays have been filled... Starting 2D histogram generation")
    
    plt.clf()
    plt.hist2d(np.array(x), np.array(y), range=[[-50, 50], [-50, 50]], 
               bins=num_dither, weights=np.array(acceptance_at_3550), vmin=0.4, vmax=0.7)
    plt.colorbar()
    plt.clim(0.4,0.7)
    plt.xlabel("X [mm]")
    plt.ylabel("Y [mm]")
    plt.suptitle("Fiber Acceptance at 3550 A\nExpected Acceptance %f\nCenter Value %f"%(expected_acceptance_at_3550, 
                                                                                        center_acceptance_at_3550))
    plt.savefig("figures/dither_fiber_acceptance_3550_%s_%d_%s.png"%(curr_desi_model_string, fiber_pos_seed, 
                                                                     append_name))
    
    plt.clf()
    plt.hist(diff_acceptance_at_3550)
    plt.xlabel("Difference in Acceptance")
    #plt.suptitle("Fiber Acceptance at 3550 A\nExpected Acceptance %f\nCenter Value %f"%(expected_acceptance_at_3550, 
    #                                                                                    center_acceptance_at_3550))
    plt.savefig("figures/dither_diff_fiber_acceptance_3550_%s_%d_%s.png"%(curr_desi_model_string, fiber_pos_seed, 
                                                                     append_name))
    
    plt.clf()
    plt.hist2d(np.array(x), np.array(y), range=[[-50, 50], [-50, 50]], 
               bins=num_dither, weights=np.array(acceptance_at_6700), vmin=0.4, vmax=0.7)
    plt.colorbar()
    plt.clim(0.4,0.7)
    plt.xlabel("X [mm]")
    plt.ylabel("Y [mm]")
    plt.suptitle("Fiber Acceptance at 6700 A\nExpected Acceptance %f\nCenter Value %f"%(expected_acceptance_at_6700, 
                                                                                        center_acceptance_at_6700))
    plt.savefig("figures/dither_fiber_acceptance_6700_%s_%d_%s.png"%(curr_desi_model_string, fiber_pos_seed, 
                                                                     append_name))
    
    plt.clf()
    plt.hist(diff_acceptance_at_6700)
    plt.xlabel("Difference in Acceptance")
    #plt.suptitle("Fiber Acceptance at 3550 A\nExpected Acceptance %f\nCenter Value %f"%(expected_acceptance_at_3550, 
    #                                                                                    center_acceptance_at_3550))
    plt.savefig("figures/dither_diff_fiber_acceptance_6700_%s_%d_%s.png"%(curr_desi_model_string, fiber_pos_seed, 
                                                                     append_name))
    
    plt.clf()
    plt.hist2d(np.array(x), np.array(y), range=[[-50, 50], [-50, 50]], 
               bins=num_dither, weights=np.array(acceptance_at_9850), vmin=0.4, vmax=0.7)
    plt.colorbar()
    plt.clim(0.4,0.7)
    plt.xlabel("X [mm]")
    plt.ylabel("Y [mm]")
    plt.suptitle("Fiber Acceptance at 9850 A\nExpected Acceptance %f\nCenter Value %f"%(expected_acceptance_at_9850, 
                                                                                        center_acceptance_at_9850))
    plt.savefig("figures/dither_fiber_acceptance_9850_%s_%d_%s.png"%(curr_desi_model_string, fiber_pos_seed, 
                                                                     append_name))
    
    plt.clf()
    plt.hist(diff_acceptance_at_9850)
    plt.xlabel("Difference in Acceptance")
    #plt.suptitle("Fiber Acceptance at 3550 A\nExpected Acceptance %f\nCenter Value %f"%(expected_acceptance_at_3550, 
    #                                                                                    center_acceptance_at_3550))
    plt.savefig("figures/dither_diff_fiber_acceptance_9850_%s_%d_%s.png"%(curr_desi_model_string, fiber_pos_seed, 
                                                                     append_name))
    
    print("Finished analysis for this configuration")
    print("You can continue with the next iteration")
    
    # TODO: this function should return some values for further evaluation
    #       besides plotting somethings

In [11]:
iterations   = 20
random_seeds = []
focal_pos    = []
for i in range(iterations):
    # fiber positions
    random_integer = np.random.randint(0, 1000)
    focal_pos.append(generate_fiber_positions(num_source, isRandom=True, seed=random_integer))
    random_seeds.append(random_integer)
np.array(random_seeds)
print("Random numbers have been generated for fiber position")
print("Fiber positions have been assigned")

Random numbers have been generated for fiber position
Fiber positions have been assigned


In [ ]:
# the source information will be used later on for the fiber loss calculation
# using their expected spectrum
qsos_round = []
elgs_round = []
lrgs_round = []
qsos_elliptic = []
elgs_elliptic = []
lrgs_elliptic = []

def generate_sources():
    for i in range(20):
        # Round sources are generated here
        qsos_round.append(generate_source(1, 0., 0., seed=23+i))
        elgs_round.append(generate_source(1, 1., 0., seed=23+i))
        lrgs_round.append(generate_source(1, 0., 1., seed=23+i))
        # elliptic objects are generated here
        qsos_elliptic.append(generate_source(num_source, 0., 0., seed=1+i, minormajor=[0.4, 0.6]))
        elgs_elliptic.append(generate_source(num_source, 1., 0., seed=1+i, minormajor=[0.4, 0.6]))
        lrgs_elliptic.append(generate_source(num_source, 0., 1., seed=1+i, minormajor=[0.4, 0.6]))
    print("round and elliptic sources has been generated")

In [47]:
for i in range(1):
    #run(random_seeds[i], *focal_pos[i], *qsos_round[0], append_name="source1")
    try:
        run(random_seeds[i], *focal_pos[i], *qsos_round[0], append_name="roundsource1")
        #run(random_seeds[i], *focal_pos[i], *qsos_elliptic[0], append_name="ellipticsource1")
        #run(random_seeds[i], *focal_pos[i], *qsos_round[1], append_name="source2")
    except:
        continue

wavelength grid has been generated
fiber position has been plotted
fiber loss calculations finished
arrays have been filled... Starting 2D histogram generation
Finished analysis for this configuration
You can continue with the next iteration


TODO:
0. Fix the colorbar range to 0.5 - 0.7 for all plots
1. Get the same plots without blur+offset, can do without offset but need to create a new blur file with no blur in it
2. Get some histograms for the difference between center value and expected value for acceptance
3. Get 2D histograms for the distance of the maximum acceptance wrt to the center value acceptance
4. 

In [17]:
qsos_round = []
for i in range(num_source):
    qsos_round.append(generate_sources(1, 0., 0., seed=i))


In [36]:
qsos_elliptic = []
elgs_elliptic = []
lrgs_elliptic = []
for i in range(20):
    qsos_elliptic.append(generate_sources(num_source, 0., 0., seed=1+i, minormajor=[0.4, 0.6]))
    elgs_elliptic.append(generate_sources(num_source, 1., 0., seed=1+i, minormajor=[0.4, 0.6]))
    lrgs_elliptic.append(generate_sources(num_source, 0., 1., seed=1+i, minormajor=[0.4, 0.6]))

wlen_grid = create_wlen_grid()
seeing_fwhm = curr_desi_model.atmosphere.get_seeing_fwhm(wlen_grid).to(u.arcsec).value
scale, blur, offset = curr_desi_model.instrument.get_focal_plane_optics(*focal_pos[0], wlen_grid)
calc = calculateFiberLoss(wlen_grid, desi=curr_desi_model)
print( calc.calculate(seeing_fwhm,
                      scale.to(u.um / u.arcsec).value, offset.to(u.um).value,
                      blur.to(u.um).value,
                      *elgs_elliptic[0]) )
print(*elgs_elliptic[0])
print( calc.calculate(seeing_fwhm,
                      scale.to(u.um / u.arcsec).value, offset.to(u.um).value,
                      blur.to(u.um).value,
                      *elgs_elliptic[1]) )
print(*elgs_elliptic[10])

[[ 0.33296477  0.3399997   0.36108438  0.38240008  0.39645283  0.40400347
   0.40833648  0.41114392  0.41276425  0.41351677  0.41313685]]
[[ 1.  0.]] [[ 0.45  1.  ]] [[ 0.4  0.6]] [[ 150.12792169  259.31681764]]
[[ 0.33392319  0.34128283  0.36203148  0.38290594  0.3966609   0.40407496
   0.40836827  0.41120453  0.4129334   0.41384312  0.41365561]]
[[ 1.  0.]] [[ 0.45  1.  ]] [[ 0.4  0.6]] [[ 64.897088     7.01108694]]
